In [ ]:
%load_ext autoreload
%autoreload 3

from general import *

import pandas as pd
import numpy as np

In [ ]:
## 1. SITE-SPECIFIC SETTINGS ##
year_start     = 1900
doy_start      = 1
NDAYS          = 40543
file_weather   = '../weather/weather_CONIFEROUS-1.txt'
file_params    = '../parameters/parameters.txt'
parcol         = 1

matrix_weather = read_weather_BASFOR( year_start, doy_start, NDAYS, file_weather )

df_params = pd.read_csv(file_params, sep="\t", header=0, index_col=0)
params = df_params.iloc[:, parcol - 1].to_numpy()

# Fill calendar_Ndep with the provided values
calendar_Ndep[0, :]  = [1700, 200, 3.0 / (365 * 10000)]
calendar_Ndep[1, :]  = [1900, 200, 3.0 / (365 * 10000)]
calendar_Ndep[2, :]  = [1910, 200, 4.7 / (365 * 10000)]
calendar_Ndep[3, :]  = [1920, 200, 6.4 / (365 * 10000)]
calendar_Ndep[4, :]  = [1930, 200, 8.9 / (365 * 10000)]
calendar_Ndep[5, :]  = [1940, 200, 12.4 / (365 * 10000)]
calendar_Ndep[6, :]  = [1950, 200, 17.2 / (365 * 10000)]
calendar_Ndep[7, :]  = [1960, 200, 23.9 / (365 * 10000)]
calendar_Ndep[8, :]  = [1970, 200, 30.9 / (365 * 10000)]
calendar_Ndep[9, :]  = [1980, 200, 35.0 / (365 * 10000)]
calendar_Ndep[10, :] = [1990, 200, 32.2 / (365 * 10000)]
calendar_Ndep[11, :] = [2000, 200, 25.1 / (365 * 10000)]
calendar_Ndep[12, :] = [2010, 200, 21.2 / (365 * 10000)]
calendar_Ndep[13, :] = [2100, 200, 21.2 / (365 * 10000)]

# Fill calendar_thinT with the provided values
calendar_thinT[0, :] = [1925, 1, 0.400]
calendar_thinT[1, :] = [1935, 1, 0.250]
calendar_thinT[2, :] = [1945, 1, 0.250]
calendar_thinT[3, :] = [1955, 1, 0.200]
calendar_thinT[4, :] = [1965, 1, 0.150]
calendar_thinT[5, :] = [1975, 1, 0.400]
calendar_thinT[6, :] = [2006, 1, 0.887]

import ctypes
import numpy as np

def call_BASFOR_C(params, matrix_weather, calendar_fert, calendar_ndep,
                  calendar_prunt, calendar_thint, ndays, nout, y):
    """
    Call the BASFOR_C Fortran routine via ctypes.
    
    Parameters:
      params         - 1D numpy array, shape (100,), dtype=np.float64
      matrix_weather - 2D numpy array, shape (ndays, 7), dtype=np.float64
      calendar_fert  - 2D numpy array, shape (100, 3), dtype=np.float64
      calendar_ndep  - 2D numpy array, shape (100, 3), dtype=np.float64
      calendar_prunt - 2D numpy array, shape (100, 3), dtype=np.float64
      calendar_thint - 2D numpy array, shape (100, 3), dtype=np.float64
      y              - 2D numpy array, shape (ndays, nout), dtype=np.float64
                       (This array must be preallocated; its contents will be
                        overwritten by BASFOR_C.)
    """

    params = np.pad(params, (0, max(0, 100 - len(params))), mode='constant', constant_values=0)
    
    # Ensure the arrays are Fortran-ordered (column-major) since Fortran expects that.
    params = np.asfortranarray(params, dtype=np.float64)
    matrix_weather = np.asfortranarray(matrix_weather, dtype=np.float64)
    calendar_fert = np.asfortranarray(calendar_fert, dtype=np.float64)
    calendar_ndep = np.asfortranarray(calendar_ndep, dtype=np.float64)
    calendar_prunt = np.asfortranarray(calendar_prunt, dtype=np.float64)
    calendar_thint = np.asfortranarray(calendar_thint, dtype=np.float64)
    y = np.asfortranarray(y, dtype=np.float64)
    
    # Load the shared library (adjust the library name/path as needed)
    lib = ctypes.CDLL("../BASFOR_conif.so")
    
    # Set the argument types for the BASFOR_C routine.
    # The Fortran subroutine accepts pointers (as c_void_p) and two integers.
    lib.BASFOR_C.argtypes = [
        ctypes.c_void_p,  # params_ptr
        ctypes.c_void_p,  # matrix_weather_ptr
        ctypes.c_void_p,  # calendar_fert_ptr
        ctypes.c_void_p,  # calendar_ndep_ptr
        ctypes.c_void_p,  # calendar_prunt_ptr
        ctypes.c_void_p,  # calendar_thint_ptr
        ctypes.c_int,     # ndays
        ctypes.c_int,     # nout
        ctypes.c_void_p   # y_ptr
    ]
    
    assert params.dtype == np.float64
    assert matrix_weather.dtype == np.float64
    assert calendar_fert.dtype == np.float64
    assert calendar_ndep.dtype == np.float64
    assert calendar_prunt.dtype == np.float64
    assert calendar_thint.dtype == np.float64
    assert y.dtype == np.float64

    def assert_eq(a, b):
        assert a == b, f"Assertion failed: {a} != {b}"

    assert_eq(params.shape, (100,))
    assert_eq(matrix_weather.shape, (NMAXDAYS, 7))
    assert_eq(calendar_fert.shape, (100, 3))
    assert_eq(calendar_ndep.shape, (100, 3))
    assert_eq(calendar_prunt.shape, (100, 3))
    assert_eq(calendar_thint.shape, (100, 3))
    assert_eq(y.shape, (NDAYS, NOUT))

    # Get pointers to the data of each array.
    params_ptr = params.ctypes.data_as(ctypes.c_void_p)
    matrix_weather_ptr = matrix_weather.ctypes.data_as(ctypes.c_void_p)
    calendar_fert_ptr = calendar_fert.ctypes.data_as(ctypes.c_void_p)
    calendar_ndep_ptr = calendar_ndep.ctypes.data_as(ctypes.c_void_p)
    calendar_prunt_ptr = calendar_prunt.ctypes.data_as(ctypes.c_void_p)
    calendar_thint_ptr = calendar_thint.ctypes.data_as(ctypes.c_void_p)
    y_ptr = y.ctypes.data_as(ctypes.c_void_p)
    
    # Call the Fortran routine. This call updates y in place.
    lib.BASFOR_C(params_ptr, matrix_weather_ptr, calendar_fert_ptr,
                 calendar_ndep_ptr, calendar_prunt_ptr, calendar_thint_ptr,
                 ndays, nout, y_ptr)
    
    # y now contains the output from BASFOR_C.
    return y


In [ ]:
y = np.zeros((NDAYS, NOUT), dtype=np.float64)
output = call_BASFOR_C(params, matrix_weather, calendar_fert, calendar_Ndep,
                           calendar_prunT, calendar_thinT, NDAYS, NOUT, y)


In [ ]:
R_output = np.loadtxt("../output_R.csv", delimiter=",")

d = NDAYS - 1

for o_idx, name in enumerate(outputNames):
    print(name)
    print(f"Python: {output[d, o_idx]}")
    print(f"R     : {R_output[d, o_idx]}")

    if R_output[d, o_idx] == 0:
        diff_percent = 0
    else:
        diff_percent = 100 * (output[d, o_idx] - R_output[d, o_idx]) / R_output[d, o_idx]
        print(f"Diff : {diff_percent:.2f}%")
    print()
